# Experiments

### Setup

In [1]:
# You can set them inline
import os
os.environ["OPENAI_API_KEY"] = ""
os.environ["LANGCHAIN_API_KEY"] = ""
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "langsmith-academy"

In [2]:
# Or you can use a .env file
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env", override=True)

True

Here is the RAG Application that we've been working with throughout this course

In [4]:
import os
import tempfile
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.sitemap import SitemapLoader
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_openai import OpenAIEmbeddings
from langsmith import traceable
from openai import OpenAI
from typing import List
import nest_asyncio

# TODO: Configure this model!
MODEL_NAME = "gpt-3.5-turbo"
MODEL_PROVIDER = "openai"
APP_VERSION = 1.0
RAG_SYSTEM_PROMPT = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.
"""

openai_client = OpenAI()

def get_vector_db_retriever():
    persist_path = os.path.join(tempfile.gettempdir(), "union.parquet")
    embd = OpenAIEmbeddings()

    # If vector store exists, then load it
    if os.path.exists(persist_path):
        vectorstore = SKLearnVectorStore(
            embedding=embd,
            persist_path=persist_path,
            serializer="parquet"
        )
        return vectorstore.as_retriever(lambda_mult=0)

    # Otherwise, index LangSmith documents and create new vector store
    ls_docs_sitemap_loader = SitemapLoader(web_path="https://docs.smith.langchain.com/sitemap.xml")
    ls_docs = ls_docs_sitemap_loader.load()

    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=500, chunk_overlap=0
    )
    doc_splits = text_splitter.split_documents(ls_docs)

    vectorstore = SKLearnVectorStore.from_documents(
        documents=doc_splits,
        embedding=embd,
        persist_path=persist_path,
        serializer="parquet"
    )
    vectorstore.persist()
    return vectorstore.as_retriever(lambda_mult=0)

nest_asyncio.apply()
retriever = get_vector_db_retriever()

"""
retrieve_documents
- Returns documents fetched from a vectorstore based on the user's question
"""
@traceable(run_type="chain")
def retrieve_documents(question: str):
    return retriever.invoke(question)

"""
generate_response
- Calls `call_openai` to generate a model response after formatting inputs
"""
@traceable(run_type="chain")
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {
            "role": "system",
            "content": RAG_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    return call_openai(messages)

"""
call_openai
- Returns the chat completion output from OpenAI
"""
@traceable(
    run_type="llm",
    metadata={
        "ls_provider": MODEL_PROVIDER,
        "ls_model_name": MODEL_NAME
    }
)
def call_openai(messages: List[dict]) -> str:
    return openai_client.chat.completions.create(
        model=MODEL_NAME,
        messages=messages,
    )

"""
langsmith_rag
- Calls `retrieve_documents` to fetch documents
- Calls `generate_response` to generate a response based on the fetched documents
- Returns the model response
"""
@traceable(run_type="chain")
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.choices[0].message.content


USER_AGENT environment variable not set, consider setting it to identify your requests.


### Experiment

Here is a code snippet that should look similar to what you see from the starter code!

There are a few important components here.

1. We have defined an Evaluator
2. We pipe our dataset examples (dict) to the shape of input that our function `langsmith_rag` takes (str) using a target function

In [6]:
from langsmith import evaluate, Client

client = Client()
dataset_name = "RAG Application Golden Dataset"

def is_concise_enough(reference_outputs: dict, outputs: dict) -> dict:
    score = len(outputs["output"]) < 1.5 * len(reference_outputs["output"])
    return {"key": "is_concise", "score": int(score)}

def target_function(inputs: dict):
    return langsmith_rag(inputs["question"])

evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="gpt-4o"
)

d:\Study\LangSmith\ls-academy\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


View the evaluation results for experiment: 'gpt-4o-30195f4d' at:
https://smith.langchain.com/o/61fc4255-5c94-438a-969e-f7649695f391/datasets/04b0d251-5d66-4bf9-9e23-6ad4dad44904/compare?selectedSessions=e536f070-bb20-48ec-8499-4e75efe2e6d4




11it [00:24,  2.24s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith when using Lang...,None,To set up tracing to LangSmith while using Lan...,1,3.114121,bf0bd19d-322a-449d-9832-28e5600e35cc,63362603-9d44-447e-9b1e-a23e8ddecb2f
1,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,1,2.925865,96c58fb4-4d66-44e6-beb1-a04b04293fc1,1490b9b6-300d-4f99-ba73-ed1063bc4296
2,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1,3.658328,f2901f92-594e-45d2-abac-6676c93fc072,c94b9d87-487c-4859-b71c-b4c2ce19755e
3,Can LangSmith be used for finetuning and model...,"LangSmith can be used for debugging, tracing, ...",None,"Yes, LangSmith can be used for fine-tuning and...",0,1.497030,46a41a14-f08f-433d-ba6e-5722045f1e12,84150bfb-6f44-4003-8f46-05cb791933d5
4,Does LangSmith support offline evaluation?,"Yes, LangSmith supports offline evaluation thr...",None,"Yes, LangSmith supports offline evaluation thr...",1,1.587221,cc22c39c-b109-421a-bb2a-10b15881ee63,9f2dc7be-1429-4419-9132-1abd3f3b9803
5,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation as a...",None,"Yes, LangSmith supports online evaluation as a...",0,2.319375,a96754a0-24b6-48e3-b160-476da83a9436,53c4e4b3-e088-4b5e-a5d6-ef265cdceae3
6,What testing capabilities does LangSmith have?,LangSmith allows developers to create datasets...,None,LangSmith offers capabilities for creating dat...,1,1.446940,ee01a8d6-0d6b-42ae-b1fa-8206ba2e75ed,e366035b-fe88-4e27-891c-aaba195ef223
7,What is LangSmith used for in three sentences?,LangSmith is a platform designed for LLM (Larg...,None,LangSmith is a platform designed for the devel...,1,1.536889,d610c585-3b87-4d5f-8902-7f0bb3065b64,a602ffec-afcc-43e6-a6f6-762f177ba2b1
8,How do I pass metadata in with @traceable?,"To pass metadata in with `@traceable`, you can...",None,You can pass metadata with the @traceable deco...,1,2.051757,935dc085-145c-4e3e-b198-f7cdb5faf3cf,d064e6db-ab05-40a1-9c70-ab6f33c9b734
9,How can I trace with the @traceable decorator?,To trace with the @traceable decorator in Pyth...,None,To trace with the @traceable decorator in Pyth...,1,2.330294,e384e146-35c6-4c75-a4d2-78a16f2ace99,809af121-7212-4d94-99f6-36f56067c655


### Modifying your Application

Now, let's change our model to gpt-35-turbo and see how it performs!

Make this change, and then run this code snippet!

In [7]:
from langsmith import evaluate, Client
from langsmith.schemas import Example, Run

def target_function(inputs: dict):
    return langsmith_rag(inputs["question"])

evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="gpt-3.5-turbo"
)

View the evaluation results for experiment: 'gpt-3.5-turbo-7a879e7d' at:
https://smith.langchain.com/o/61fc4255-5c94-438a-969e-f7649695f391/datasets/04b0d251-5d66-4bf9-9e23-6ad4dad44904/compare?selectedSessions=35edd156-00cb-4bcc-a3e4-218272c78882




11it [00:23,  2.16s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith when using Lang...,None,To set up tracing to LangSmith while using Lan...,1,2.708538,bf0bd19d-322a-449d-9832-28e5600e35cc,e31428f3-e132-478a-b5aa-5db6c6ae3efa
1,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,1,1.800448,96c58fb4-4d66-44e6-beb1-a04b04293fc1,b5f5fa6d-b0e0-4916-a979-1930d9a2bf2a
2,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents ...",None,"Yes, LangSmith can be used to evaluate agents....",1,2.803148,f2901f92-594e-45d2-abac-6676c93fc072,0a2a9b9a-d6b6-4e88-8c9b-81e6a12c3b2f
3,Can LangSmith be used for finetuning and model...,"Yes, LangSmith can be used for prototyping and...",None,"Yes, LangSmith can be used for fine-tuning and...",0,2.381916,46a41a14-f08f-433d-ba6e-5722045f1e12,1c40a05c-9b10-47c8-afa1-b04927463ca1
4,Does LangSmith support offline evaluation?,"Yes, LangSmith supports offline evaluation thr...",None,"Yes, LangSmith supports offline evaluation thr...",1,2.361019,cc22c39c-b109-421a-bb2a-10b15881ee63,6d0c78e0-2ef2-4d8e-a6bf-4f2dd4f0dac5
5,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation whic...",None,"Yes, LangSmith supports online evaluation as a...",0,2.236687,a96754a0-24b6-48e3-b160-476da83a9436,6c6bb142-092d-4278-bbb4-d7324b87602e
6,What testing capabilities does LangSmith have?,LangSmith enables developers to create dataset...,None,LangSmith offers capabilities for creating dat...,1,1.849645,ee01a8d6-0d6b-42ae-b1fa-8206ba2e75ed,f0f8f7d1-515f-4126-92b8-d87491543ac1
7,What is LangSmith used for in three sentences?,LangSmith is a platform designed for LLM (Larg...,None,LangSmith is a platform designed for the devel...,1,1.701478,d610c585-3b87-4d5f-8902-7f0bb3065b64,67dc2df9-4b75-48bb-ad71-790e3ee89c58
8,How do I pass metadata in with @traceable?,"To pass metadata with `@traceable`, you would ...",None,You can pass metadata with the @traceable deco...,1,2.229776,935dc085-145c-4e3e-b198-f7cdb5faf3cf,887a48a6-c6b6-4a19-bc4c-f99c6fb7bf16
9,How can I trace with the @traceable decorator?,To trace with the @traceable decorator in the ...,None,To trace with the @traceable decorator in Pyth...,1,1.603318,e384e146-35c6-4c75-a4d2-78a16f2ace99,79d9ef4e-b5c6-4b23-a93d-516a4ca2fe34


### Running over Different pieces of Data

##### Dataset Version

You can execute an experiment on a specific version of a dataset in the sdk by using the `as_of` parameter in `list_examples`

Let's try running on just our initial dataset.

In [9]:
evaluate(
    target_function,
    data=client.list_examples(dataset_name=dataset_name, as_of="Initial dataset"),   # We use as_of to specify a version
    evaluators=[is_concise_enough],
    experiment_prefix="initial dataset version"
)

View the evaluation results for experiment: 'initial dataset version-9b76d5f2' at:
https://smith.langchain.com/o/61fc4255-5c94-438a-969e-f7649695f391/datasets/04b0d251-5d66-4bf9-9e23-6ad4dad44904/compare?selectedSessions=9c8dd873-a03a-4d2a-8757-18c306ba962d




10it [00:20,  2.08s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,1,3.035065,96c58fb4-4d66-44e6-beb1-a04b04293fc1,d8164a23-9f15-44b7-b64b-7dc972721c5d
1,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents ...",None,"Yes, LangSmith can be used to evaluate agents....",0,2.152864,f2901f92-594e-45d2-abac-6676c93fc072,794e8125-f6e7-4588-b6f1-06e17a8c0c57
2,Can LangSmith be used for finetuning and model...,LangSmith is primarily focused on LLM applicat...,None,"Yes, LangSmith can be used for fine-tuning and...",1,2.462826,46a41a14-f08f-433d-ba6e-5722045f1e12,c355e5c0-1321-4b8b-9dba-94c40378e3a8
3,Does LangSmith support offline evaluation?,"Yes, LangSmith supports offline evaluation thr...",None,"Yes, LangSmith supports offline evaluation thr...",1,1.582810,cc22c39c-b109-421a-bb2a-10b15881ee63,9db170fb-4e72-4317-aa12-e773b3ec5df9
4,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation as a...",None,"Yes, LangSmith supports online evaluation as a...",0,1.732069,a96754a0-24b6-48e3-b160-476da83a9436,7e22aa68-3375-4dfb-8ea8-ca404288484f
5,What testing capabilities does LangSmith have?,LangSmith allows developers to create datasets...,None,LangSmith offers capabilities for creating dat...,1,1.989763,ee01a8d6-0d6b-42ae-b1fa-8206ba2e75ed,9ea097fd-0724-4a65-a7fc-544de77adbf3
6,What is LangSmith used for in three sentences?,LangSmith is a platform designed for LLM appli...,None,LangSmith is a platform designed for the devel...,1,1.457586,d610c585-3b87-4d5f-8902-7f0bb3065b64,7e724bbc-e11c-4057-ae41-167ade170cda
7,How do I pass metadata in with @traceable?,"To pass metadata with `@traceable`, you need t...",None,You can pass metadata with the @traceable deco...,1,2.082338,935dc085-145c-4e3e-b198-f7cdb5faf3cf,474a04b6-618f-4011-bc9a-f67fca648499
8,How can I trace with the @traceable decorator?,To trace with the @traceable decorator in Pyth...,None,To trace with the @traceable decorator in Pyth...,1,1.848779,e384e146-35c6-4c75-a4d2-78a16f2ace99,725a33e2-99b4-4016-8692-714039508f00
9,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith while using Lan...,None,To set up tracing to LangSmith while using Lan...,0,1.897342,80ede8b0-1569-4ca2-a5cf-5167f02e5ec7,739c9763-9d1a-4b85-b49c-2dde4b116f25


##### Dataset Split

You can run an experiment on a specific split of your dataset, let's try running on the Crucial Examples split.

In [ ]:
evaluate(
    target_function,
    data=client.list_examples(dataset_name=dataset_name, splits=["Crucial Examples"]),  # We pass in a list of Splits
    evaluators=[is_concise_enough],
    experiment_prefix="Crucial Examples split"
)

##### Specific Data Points

You can specify individual data points to run an experiment over as well

In [ ]:
evaluate(
    target_function,
    data=client.list_examples(
        dataset_name=dataset_name, 
        example_ids=[   # We pass in a specific list of example_ids
            # TODO: You will need to paste in your own example ids for this to work!
            "",
            ""
        ]
    ),
    evaluators=[is_concise_enough],
    experiment_prefix="two specific example ids"
)

### Other Parameters

##### Repetitions

You can run an experiment several times to make sure you have consistent results

In [10]:
evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="two repetitions",
    num_repetitions=2   # This field defaults to 1
)

View the evaluation results for experiment: 'two repetitions-b30be9c5' at:
https://smith.langchain.com/o/61fc4255-5c94-438a-969e-f7649695f391/datasets/04b0d251-5d66-4bf9-9e23-6ad4dad44904/compare?selectedSessions=39e1c3dc-dff0-4fb6-b304-227475517053




22it [00:45,  2.05s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith when using Lang...,None,To set up tracing to LangSmith while using Lan...,1,2.985509,bf0bd19d-322a-449d-9832-28e5600e35cc,de47b829-89d3-406d-9733-fe3a1da69c02
1,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,1,2.145600,96c58fb4-4d66-44e6-beb1-a04b04293fc1,ce8d3203-020f-461f-b6ac-17534bc34ee7
2,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents ...",None,"Yes, LangSmith can be used to evaluate agents....",1,2.081307,f2901f92-594e-45d2-abac-6676c93fc072,a980554b-5fa8-4cd1-89e3-51856e2fd403
3,Can LangSmith be used for finetuning and model...,LangSmith can be used for prototyping LLM appl...,None,"Yes, LangSmith can be used for fine-tuning and...",0,3.349625,46a41a14-f08f-433d-ba6e-5722045f1e12,89a78181-0424-462e-a15e-73f90f75fb19
4,Does LangSmith support offline evaluation?,"Yes, LangSmith supports offline evaluation thr...",None,"Yes, LangSmith supports offline evaluation thr...",1,1.646951,cc22c39c-b109-421a-bb2a-10b15881ee63,63cbb076-bcd1-4b28-ac21-cbb657dbf409
5,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation as a...",None,"Yes, LangSmith supports online evaluation as a...",0,2.230137,a96754a0-24b6-48e3-b160-476da83a9436,1a9430fa-78ed-4461-a073-8e8c7ba2f004
6,What testing capabilities does LangSmith have?,LangSmith allows developers to create datasets...,None,LangSmith offers capabilities for creating dat...,1,1.336872,ee01a8d6-0d6b-42ae-b1fa-8206ba2e75ed,5ab3aa94-82c9-4b57-b9d3-0b61b99812c9
7,What is LangSmith used for in three sentences?,LangSmith is a platform designed for LLM appli...,None,LangSmith is a platform designed for the devel...,1,3.937334,d610c585-3b87-4d5f-8902-7f0bb3065b64,7b699c0c-d4eb-4dfb-9c97-b6f2ed243557
8,How do I pass metadata in with @traceable?,"To pass metadata with `@traceable`, you need t...",None,You can pass metadata with the @traceable deco...,1,1.573946,935dc085-145c-4e3e-b198-f7cdb5faf3cf,c781cd61-cd46-4756-be97-855aa1597714
9,How can I trace with the @traceable decorator?,"To trace with the @traceable decorator, you ne...",None,To trace with the @traceable decorator in Pyth...,1,1.920311,e384e146-35c6-4c75-a4d2-78a16f2ace99,cdac57b8-9e5f-4a98-a97a-c3dd9b2b50e8


##### Concurrency
You can also kick off concurrent threads of execution to make your experiments finish faster!

In [11]:
evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="concurrency",
    max_concurrency=3,  # This defaults to None, so this is an improvement!
)

View the evaluation results for experiment: 'concurrency-08622047' at:
https://smith.langchain.com/o/61fc4255-5c94-438a-969e-f7649695f391/datasets/04b0d251-5d66-4bf9-9e23-6ad4dad44904/compare?selectedSessions=2cbc92d7-0d41-4323-8fc1-be9a04222680




11it [00:07,  1.41it/s]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith while using Lan...,None,To set up tracing to LangSmith while using Lan...,1,1.700875,bf0bd19d-322a-449d-9832-28e5600e35cc,23b8c71b-98f2-4c2c-a5c0-b0f541fea012
1,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents ...",None,"Yes, LangSmith can be used to evaluate agents....",1,2.176565,f2901f92-594e-45d2-abac-6676c93fc072,426d0038-619c-491b-90dd-7d6ce5a9800f
2,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,1,3.031793,96c58fb4-4d66-44e6-beb1-a04b04293fc1,a0bbe386-6b40-474e-845f-f8d6c2be4221
3,Can LangSmith be used for finetuning and model...,LangSmith can be used for prototyping and debu...,None,"Yes, LangSmith can be used for fine-tuning and...",1,1.375631,46a41a14-f08f-433d-ba6e-5722045f1e12,cc9376d2-35a6-41fc-a7fd-83b72b367e58
4,Does LangSmith support offline evaluation?,"Yes, LangSmith supports offline evaluation thr...",None,"Yes, LangSmith supports offline evaluation thr...",1,1.663322,cc22c39c-b109-421a-bb2a-10b15881ee63,49514c6c-84dd-48f5-8922-27c178bf59c7
5,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation as a...",None,"Yes, LangSmith supports online evaluation as a...",1,1.483598,a96754a0-24b6-48e3-b160-476da83a9436,090dbcac-0ed2-49a4-91e2-ba03cf083f9c
6,What testing capabilities does LangSmith have?,LangSmith allows developers to create datasets...,None,LangSmith offers capabilities for creating dat...,1,1.606522,ee01a8d6-0d6b-42ae-b1fa-8206ba2e75ed,dfbafd6c-50d7-4b01-ae1a-5e57cf0f0a4c
7,What is LangSmith used for in three sentences?,LangSmith is used as a platform for building a...,None,LangSmith is a platform designed for the devel...,1,1.484905,d610c585-3b87-4d5f-8902-7f0bb3065b64,08658013-a965-46e4-8a51-07223ac6f5f2
8,How do I pass metadata in with @traceable?,"To pass metadata in with `@traceable`, you nee...",None,You can pass metadata with the @traceable deco...,0,1.874667,935dc085-145c-4e3e-b198-f7cdb5faf3cf,9e7aa681-2fb6-456e-939d-65d1c51568bb
9,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith when using Lang...,None,To set up tracing to LangSmith while using Lan...,0,1.646842,80ede8b0-1569-4ca2-a5cf-5167f02e5ec7,42b84d5b-0974-4efa-ae0e-c761c7bcabd3


##### Metadata 

You can (and should) add metadata to your experiments, to make them easier to find in the UI

In [12]:
evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="metadata added",
    metadata={  # We can pass custom metadata for the experiment, such as the model name
        "model_name": MODEL_NAME
    }
)

View the evaluation results for experiment: 'metadata added-6f20644b' at:
https://smith.langchain.com/o/61fc4255-5c94-438a-969e-f7649695f391/datasets/04b0d251-5d66-4bf9-9e23-6ad4dad44904/compare?selectedSessions=90655b65-8736-416c-b7f4-bfb6e77003d1




11it [00:18,  1.71s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith while using Lan...,None,To set up tracing to LangSmith while using Lan...,1,1.601078,bf0bd19d-322a-449d-9832-28e5600e35cc,caa3d1d0-17fb-4882-8f86-56e088003202
1,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,1,1.728852,96c58fb4-4d66-44e6-beb1-a04b04293fc1,e8bd3d24-3890-4637-bba2-6ceb163441d0
2,Can LangSmith be used to evaluate agents?,LangSmith can be used to evaluate agents throu...,None,"Yes, LangSmith can be used to evaluate agents....",1,1.578752,f2901f92-594e-45d2-abac-6676c93fc072,8f34ca9a-2f4f-46d5-aa01-16a3ce3b71ad
3,Can LangSmith be used for finetuning and model...,LangSmith can be used for prototyping LLM appl...,None,"Yes, LangSmith can be used for fine-tuning and...",0,2.062767,46a41a14-f08f-433d-ba6e-5722045f1e12,83252084-7321-4a94-9075-c60f0ccf8a6c
4,Does LangSmith support offline evaluation?,"Yes, LangSmith supports offline evaluation thr...",None,"Yes, LangSmith supports offline evaluation thr...",1,1.674874,cc22c39c-b109-421a-bb2a-10b15881ee63,c38b72bd-78f1-4903-9e2f-5c5c65026898
5,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation. It ...",None,"Yes, LangSmith supports online evaluation as a...",1,1.442355,a96754a0-24b6-48e3-b160-476da83a9436,4c65359f-10e8-466f-bf1b-ed6d835a002a
6,What testing capabilities does LangSmith have?,LangSmith allows developers to create datasets...,None,LangSmith offers capabilities for creating dat...,1,1.568932,ee01a8d6-0d6b-42ae-b1fa-8206ba2e75ed,c6b0d62c-2ea0-4846-b8d3-8e94ca33ae2c
7,What is LangSmith used for in three sentences?,"LangSmith is a platform used for developing, m...",None,LangSmith is a platform designed for the devel...,1,1.507838,d610c585-3b87-4d5f-8902-7f0bb3065b64,be0ca3d1-6384-4f7a-b641-379faf88532b
8,How do I pass metadata in with @traceable?,To pass metadata along with traces using @trac...,None,You can pass metadata with the @traceable deco...,1,1.891144,935dc085-145c-4e3e-b198-f7cdb5faf3cf,cd48c7e2-05f5-4ada-94e1-52db0513ffd3
9,How can I trace with the @traceable decorator?,To trace with the @traceable decorator in Pyth...,None,To trace with the @traceable decorator in Pyth...,1,1.717507,e384e146-35c6-4c75-a4d2-78a16f2ace99,f1c03f65-8dbf-4323-b34b-d91942f00701
